# Particle Swarm Optimization


# Import and Setup

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import os

# Create plots
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

# Random seed for reproducibility
np.random.seed(42)

# Creates folders for outputs
os.makedirs('plots', exist_ok=True)
os.makedirs('results', exist_ok=True)

print("Setup complete!")
print(f"NumPy version: {np.__version__}")

Setup complete!
NumPy version: 1.26.3


In [27]:
# Base class for test functions
# This way we don't repeat code for each function

class TestFunction:
    def __init__(self, name, bounds_lower, bounds_upper):
        self.name = name
        self.bounds_lower = bounds_lower
        self.bounds_upper = bounds_upper
        self.n_evaluations = 0  # keep track of how many times we call the function
    
    def evaluate(self, x):
        # Each specific function will implement this
        raise NotImplementedError("Need to implement this in subclass")
    
    def __call__(self, x):
        # This lets us call the function like func(x) instead of func.evaluate(x)
        return self.evaluate(x)
    
    def reset_counter(self):
        self.n_evaluations = 0

print("Base class ready")

Base class ready


# Sphere Function

In [ ]:
class SphereFunction(TestFunction):
    """Sphere function"""
    def __init__(self):
        super().__init__("Sphere", -5.12, 5.12)
    
    def evaluate(self, x):
        self.n_evaluations += 1
        return np.sum(x ** 2)


class RosenbrockFunction(TestFunction):
    """Rosenbrock benchmark function"""
    def __init__(self):
        super().__init__("Rosenbrock", -5.0, 10.0)
    
    def evaluate(self, x):
        self.n_evaluations += 1
        return np.sum(100.0 * (x[1:] - x[:-1]**2)**2 + (1 - x[:-1])**2)


class RastriginFunction(TestFunction):
    """Rastrigin"""
    def __init__(self):
        super().__init__("Rastrigin", -5.12, 5.12)
    
    def evaluate(self, x):
        self.n_evaluations += 1
        n = len(x)
        return 10 * n + np.sum(x**2 - 10 * np.cos(2 * np.pi * x))


class AckleyFunction(TestFunction):
    """Ackley function"""
    def __init__(self):
        super().__init__("Ackley", -32.768, 32.768)
    
    def evaluate(self, x):
        self.n_evaluations += 1
        n = len(x)
        sum_sq = np.sum(x ** 2)
        sum_cos = np.sum(np.cos(2 * np.pi * x))
        term1 = -20 * np.exp(-0.2 * np.sqrt(sum_sq / n))
        term2 = -np.exp(sum_cos / n)
        return term1 + term2 + 20 + np.e


def get_function(name):
    """Get test function by name"""
    functions = {
        'sphere': SphereFunction(),
        'rosenbrock': RosenbrockFunction(),
        'rastrigin': RastriginFunction(),
        'ackley': AckleyFunction()
    }
    if name.lower() not in functions:
        raise ValueError(f"Unknown function: {name}")
    return functions[name.lower()]


# Quick validation
print("Test Functions Implemented:")
print("-" * 60)
for name in ['sphere', 'rosenbrock', 'rastrigin', 'ackley']:
    func = get_function(name)
    val = func(np.zeros(5))
    print(f"{func.name:12s}: f(0,0,0,0,0) = {val:10.6f}")
print("-" * 60)
print("All functions ready!")

Test Functions Implemented:
------------------------------------------------------------
Sphere      : f(0,0,0,0,0) =   0.000000
Rosenbrock  : f(0,0,0,0,0) =   4.000000
Rastrigin   : f(0,0,0,0,0) =   0.000000
Ackley      : f(0,0,0,0,0) =   0.000000
------------------------------------------------------------
All functions ready!
